<a href="https://colab.research.google.com/github/MPrazzoli/corpsol/blob/main/Script_Schedule_Date.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
pip install QuantLib

     |████████████████████████████████| 20.1MB 1.3MB/s 


In [14]:
import os
import math
import numpy as np
import pandas as pd
from QuantLib import *
from datetime import date

*Class: Market Data*

In [ ]:
class historical_fixed_rate(object):

    def __init__(self, name, reference_date = None, currency = None, underlying = None):
        
        try:
            self.reference_date = datetime.strptime(reference_date, '%d/%m/%Y')
        except:
            self.reference_date = None
        
        self.name = name
        self.currency = currency
        self.underlying = underlying
        self.historical_fixed_rate_df = pd.DataFrame

class zero_curve_rate(object):
# chiedere a Mirko l'assegnazione di interp_rule e day_count
    def __init__(self, name, reference_date = None, currency = None, underlying = None, interp_rule = None, day_count = None):
        
        self.name = name
        
        try:
            self.reference_date = datetime.strptime(reference_date, '%d/%m/%Y')
        except:
            self.reference_date = None
      
        self.currency = currency
        self.underlying = underlying
        self.interpolation_rule = interp_rule
        self.day_count = day_count
        self.zero_curve_rate_df = pd.DataFrame

    def set_interpolation(self, interpolation_rule, day_count):
        self.interpolation_rule = interpolation_rule
        self.day_count = day_count

    def add_discount(self, date, value, label):
        try:
            date = datetime.strptime(date, '%d/%m/%Y')
            self.discount = self.discount.append({'DATE': date, 'DISCOUNT': value, 'LABEL': label}, ignore_index=True)

    def set_discounts(self, discounts):
        try:
            self.discount = pd.DataFrame(discounts)
            self.discount.discount = self.discount.discount.apply(lambda x: x['value']['mid'])
            self.discount['date'] = pd.to_datetime(self.discount['date'], format='%d/%m/%Y')
            self.discount.drop('counter', axis=1, inplace=True)
            self.discount.rename(columns={'date': 'DATE', 'discount': 'DISCOUNT', 'label': 'LABEL'}, inplace=True)
            self.discount.sort_values('DATE', ascending=True, inplace=True)
            self.discount.drop(self.discount.index[0], inplace=True)

    def compute_rate(self, valuation_date, mul=1):
       
        try:
            self.discount['RATE'] = -np.log(self.discount['DISCOUNT']) / (
                    (self.discount['DATE'] - valuation_date).dt.days / 365) * mul
      
class volatility_surface(object):

    def __init__(self, name, reference_date):
        self.name = name
        try:
            self.reference_date = datetime.strptime(reference_date, '%Y-%m-%d')
      
        self.volatility_surface = pd.DataFrame()
        self.shift = None
        self.maturity_list = []
        self.strike_list = []

    def add_vola(self, maturity, shift, strike, vola):

        self.maturity_list.append(maturity)

        if self.shift is None:
            self.shift = shift

        if len(self.strike_list) == 0:
            self.strike_list.extend(strike)

        self.vola = self.vola.append(vola, ignore_index=True)

*Class: IRS & Derivative (Cap / Floor)*

In [ ]:
class IRS:

  def __init__(self):

    self.start_date = None
    self.maturity_date = None
    self.tenor = None
    self.business_convention = None
    self.date_gen_rule = None
    self.schedule = pd.DataFrame
    self.eom = None

    
    self.npv = None

class DERIVATIVE:

  def __init__(self):

    self.strike = None
    self.premium = None
    self.flows = pd.DataFrame

*Oggetto: *

In [ ]:
class DERIVATIVE:

  def __init__(self, strike):

    self.strike = None
    self.premium = None
    self.flows = pd.DataFrame

*Submit function*

In [6]:
def market_data(path):
  ############## input MX3 #######################################################

  historical_fixed_rates = historical_fixed_rate(name="HISTORICAL RATE")
  historical_fixed_rates.historical_fixed_rate_df = pd.read_excel(path, sheet_name='fixings')

  estimation_curve = zero_curve_rate(name='ESTIMATION')
  estimation_curve.zero_curve_rate_df = pd.read_excel(path, sheet_name='estimation')

  discount_curve = zero_curve_rate(name='DISCOUNT')
  discount_curve.zero_curve_rate_df = pd.read_excel(path, sheet_name='discount')

  market_data_dict = {'ESTIMATION': estimation_curve, 'DISCOUNT': discount_curve, 'HISTORICAL': historical_fixed_rates}

  return(market_data_dict)


In [11]:
calendar=TARGET()

NameError: ignored

In [10]:
def pricer(path,
           underlying,
           amortization_type,
           notional,
           amortization_rate,
           maturity_label,
           ref_date,
           tenor,
           business_convention,
           date_gen_rule,
           currency = 'EUR',
           calendar = TARGET(),
           end_of_month = False,
           deduction_formula = '-1M LBD/-1BD',
           day_count = Actual360):

    dictionary = market_data(path)
    irs = IRS()
    irs.start_date = calendar.advance(ref_date,2,Days)
    if maturity_label[-1] == 'M':
      irs.maturity_date = calendar.advance(irs.start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
    elif maturity_label[-1] == 'Y':
      irs.maturity_date = calendar.advance(irs.start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
    irs.business_convention = business_convention
    irs.tenor = tenor
    irs.date_gen_rule = date_gen_rule
    
# aggiungere altri attribuiti all'irs(calendario,EOM, deduction formula, day count)


    irs.eom = end_of_month
    irs.schedule = pd.DataFrame({'startdate':list(Schedule(irs.start_date,irs.maturity_date,irs.tenor,calendar,irs.business_convention,irs.business_convention,irs.date_gen_rule,irs.eom))[:-1],'enddate':list(Schedule(irs.start_date,irs.maturity_date,irs.tenor,calendar,irs.business_convention,irs.business_convention,irs.date_gen_rule,irs.eom))[1:]})

NameError: ignored

In [ ]:
try:
  underlying = input('Inserire il sottostante: ')
  if underlying == '':
    underlying = 'EUR3M'
except:
  underlying = 'EUR3M'

try:
  amortization_type = input('Inserire il piano di ammortamento: ')
  if amortization_type == '':
    amortization_type = 'Bullet'
except:
  amortization_type = 'Bullet'

try:
  notional = float(input('Inserire il nozionale: '))
except:
  notional = 100000.0

try:
  amortization_rate = float(input('Inserire il tasso annuo: '))
except:
  amortization_rate = .05

try:
  maturity_label = input('Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): ')
  if maturity_label == '':
     maturity_label = '5Y'
except:
  maturity_label = '5Y'

try:
  ref_date = Date(input('Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: '),'%d/%m/%Y')
  if ref_date == '':
    ref_date = Date(date.today().day,date.today().month,date.today().year)
except:
  ref_date = Date(date.today().day,date.today().month,date.today().year)

try:
  tenor = int(input('Inserire il periodo che intercorre tra le rate in numero di mesi: '))
except:
  tenor = 3

try:
  business_convention = input('Inserire la convenzione:')
  if business_convention == '' or business_convention == 'ModifiedFollowing':
    business_convention = ModifiedFollowing
except:
  business_convention = ModifiedFollowing

try:
  date_gen_rule = input('Inserire la regola per il calcolo delle date:')
  if date_gen_rule == '' or date_gen_rule == 'Forward':
    date_gen_rule = DateGeneration.Forward
  else: 
    date_gen_rule = DateGeneration.Backward
except:
  date_gen_rule = DateGeneration.Forward

market_data_path = '/content/drive/MyDrive/data.xlsx'


Inserire il sottostante: 
Inserire il piano di ammortamento: 
Inserire il nozionale: 
Inserire il tasso annuo: 
Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): 
Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: 
Inserire il periodo che intercorre tra le rate in numero di mesi: 
Inserire la convenzione:
Inserire la regola per il calcolo delle date:


In [ ]:
lc=YieldTermStructureHandle(FlatForward(1,TARGET(),0.02,Actual365Fixed()))
libor=USDLibor(Tenor,lc)

In [ ]:
fixed_rates = pd.read_excel(market_data_path, sheet_name='fixings')
estimation_rate = pd.read_excel(market_data_path, sheet_name='estimation')
discount_rate = pd.read_excel(market_data_path, sheet_name='discount')
irs_swap=VanillaSwap(VanillaSwap.Payer,notional,schedule,0.025,Actual360(),schedule,libor,0.004,Actual360())
engine=DiscountingSwapEngine(lc)
irs_swap.setPricingEngine(engine)
irs_swap.NPV()
type(discount_rate)


In [ ]:
pricer(path = market_data_path, underlying = underlying, amortization_type = amortization_type, notional = notional, amortization_rate = amortization_rate, maturity_label = maturity_label, ref_date = ref_date, tenor = tenor,
       business_convention = business_convention, date_gen_rule = date_gen_rule)

In [ ]:
#Trasformazione della durata da anni.mesi in mesi
Durata_mesi=int(int(durata)*12+round(durata-int(durata),3)*100) 
Durata_mesi

*Submit for pricing*

In [ ]:
submit(curve=curve, vola=vola, reference_date=ref, start_date=start, maturity_date=end, currency=ccy, notional=notional,
           tenor=tenor, fixing_frequency=firxing_freq, payment_calendar=paycal, fixing_calendar=fixcal, end_of_month=eom,
           business_convention=bc, termination_business_convention=tc, date_generation=dg, deduction_formula=df, day_count=daycount,
           cap_floor=derivative, cap_strike=cap_strike, floor_strike=floor_strike)

Inserimento degli input

In [ ]:
Piano=input('Inserire il piano di ammortamento: ')
nozionale=float(input('Inserire il nozionale: '))
tasso=float(input('Inserire il tassuo annuo: '))
durata=float(input('Inserire la durata del finanziamento in anni.mesi indicando sempre 2 cifre per i mesi: '))
RefDate=input('Inserire la data iniziale del finanziamento utilizzando \"/": ')
periodo=int(input('Inserire il periodo che intercorre tra le rate in numero di mesi: '))
Convenzione=input('Inserire la convezione:')
Regola=input('Inserire la regola per il calcolo delle date:')

In [ ]:
#Trasformazione della durata da anni.mesi in mesi
Durata_mesi=int(int(durata)*12+round(durata-int(durata),3)*100) 
Durata_mesi

Generazione dello schedule delle date

In [ ]:
calendar=TARGET()
RefDate_data=Date(RefDate,'%d/%m/%Y')
StartDate=calendar.advance(RefDate_data,2,Days)
terminationDate=calendar.advance(StartDate,Durata_mesi,Months)
tenor=Period(periodo,Months)
if (Convenzione == 'ModifiedFollowing'):
  convention=1
elif (Convenzione == 'ModifiedFollowing'):
    convention=0
terminationDateConvention= convention
if (Regola == 'Forward'):
  rule=1
elif (Regola == 'Backward'):
  rule=0
endofmonth=False
schedule=Schedule(StartDate,terminationDate,tenor,calendar,convention,terminationDateConvention,rule,endofmonth)
schedule_end=Schedule(StartDate+tenor,terminationDate,tenor,calendar,convention,terminationDateConvention,rule,endofmonth)
lista_startdate=pd.DataFrame({'startdate':list(schedule)})
lista_enddate=pd.DataFrame({'enddate':list(schedule_end)})

Ammortamento Francese

In [ ]:
#Calcolo della rata 
rate_totali=int((12/periodo)*round(Durata_mesi/12,2))
t=(1+tasso/(12/periodo))**rate_totali
rata=round(nozionale*t*(tasso/(12/periodo))/(t-1),2)
Residuo=nozionale
tot_I=0
Estinto=0
plan=np.array
plan=[[0,0,0,0,0,Residuo]]

In [ ]:
rate=range(0,rate_totali)
for count in rate:
  Int_quota=round(Residuo*tasso/(12/periodo),2)
  Cap_quota=(rata-Int_quota)
  tot_I=(tot_I+Int_quota)
  Residuo=(Residuo-Cap_quota)
  Estinto=(Estinto+Cap_quota)
  Tot=(Int_quota+Cap_quota)
  plan.append([Int_quota,Cap_quota,Tot,tot_I,Estinto,Residuo])

In [ ]:
index=['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
df=pd.DataFrame(plan)
df.columns=index
df1=pd.concat([lista_startdate,df],axis=1)
df2=pd.concat([lista_startdate,lista_enddate,df['Debito Residuo']],axis=1)

Ammortamento Italiano

In [ ]:
Residuo_ita=nozionale
tot_I_ita=0
Estinto_ita=0
Tot_ita=0
plan_ita=np.array
plan_ita=[[0,0,0,0,0,Residuo_ita]]

In [ ]:
rate_ita=range(0,rate_totali)
for count in rate_ita:
  Int_quota_ita=round(Residuo_ita*tasso/(12/periodo),2)
  Cap_quota_ita=round((nozionale/rate_totali),2)
  tot_I_ita=(tot_I_ita+Int_quota_ita)
  Residuo_ita=(Residuo_ita-Cap_quota_ita)
  Estinto_ita=(Estinto_ita+Cap_quota_ita)
  Tot_ita=(Int_quota_ita+Cap_quota_ita)
  plan_ita.append([Int_quota_ita,Cap_quota_ita,Tot_ita,tot_I_ita,Estinto_ita,Residuo_ita])

In [ ]:
index=['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
df_ita=pd.DataFrame(plan_ita)
df_ita.columns=index
df1_ita=pd.concat([lista_startdate,df_ita],axis=1)
df2_ita=pd.concat([lista_startdate,lista_enddate,df_ita['Debito Residuo']],axis=1)

Ammortamento Bullet

In [ ]:
Residuo_bullet=nozionale
tot_I_bullet=0
plan_bullet=np.array
plan_bullet=[[0,0,Residuo_bullet]]
rate_bullet=range(0,rate_totali)
for count in rate_bullet:
  Int_quota_bullet=round(Residuo_bullet*tasso/(12/periodo),2)
  tot_I_bullet=(tot_I_bullet+Int_quota_bullet)
  plan_bullet.append([Int_quota_bullet,tot_I_bullet,Residuo_bullet])

In [ ]:
index=['Quota Interesse','Interessi Totali','Debito Residuo']
df_bullet=pd.DataFrame(plan_bullet)
df_bullet.columns=index
df1_bullet=pd.concat([lista_startdate,df_bullet],axis=1)
df2_bullet=pd.concat([lista_startdate,lista_enddate,df_bullet['Debito Residuo']],axis=1)
df2_bullet=df2_bullet[:-1]

Output del modello

In [ ]:
if (Piano== 'Francese'):
  print (df2)
elif (Piano == 'Italiano'):
  print (df2_ita)
elif (Piano == 'Bullet'):
  print (df2_bullet)